In [ ]:
% load_ext autoreload
% autoreload 2

%matplotlib inline

In [ ]:
from dataset import *
from model import *

In [ ]:
data_test_path = os.path.join("..", "data", "test", "SRAD2018_Test_1")
data_test = srad_dataset(data_test_path, is_train=False)

In [ ]:
model = BaselineModel(1, 1, 6, batch_first=True)
model.load_state_dict(torch.load("w.h5"))

In [ ]:
import queue
import threading

trans = transforms.Compose([
    BatchApply(BatchApply(transforms.ToPILImage())),
    BatchApply(BatchApply(transforms.Resize(501))),
])

def save_seq_func(q):
    def save_seq():
        while True:
            item = q.get()
            if item is None:
                break
            seq_idx, bseq = item
            
            x = bseq
            mask = (x > 0.40).to(torch.float32)
            x = torch.clamp(x, max=0.31)
            x = x * (1 - mask) + mask
            bseq = x
            bseq = trans(bseq)
            
            for i, seq in enumerate(bseq):
                seq_id = data_test.dataset.get_id(seq_idx + i)
                path = os.path.join(result_path, seq_id)
                if not os.path.exists(path):
                    os.makedirs(path)
                for j, img in enumerate(seq):
                    filename = "{}_f{:03d}.png".format(seq_id, j + 1)
                    img.save(os.path.join(path, filename))
            q.task_done()
            print(seq_idx, "ok")
        print("done")
    return save_seq

num_workers = 8
q = queue.Queue(maxsize=128)
threads = []
for i in range(num_workers):
    t = threading.Thread(target=save_seq_func(q))
    t.start()
    threads.append(t)

In [ ]:
import torch.utils.data as data
import torchvision.transforms as transforms

result_path = os.path.join("..", "data", "test", "result")

model = model.cuda()


def predict(dataset, q):
    loader = data.DataLoader(dataset, batch_size=16)
    i = 0
    for x in loader:
        x = model(x.cuda()).cpu()
        q.put((i, x))
        print(i)
        i += len(x)
        
with torch.no_grad():
    predict(data_test, q)

In [ ]:
q.join()

for i in range(num_workers):
    q.put(None)
for t in threads:
    t.join()